# This script creates 3 Kepler GL maps. One with the top 100 busiest stations, one with population density as the background layer and one with median household income.

In [1]:
import pandas as pd
import numpy as np
from keplergl import KeplerGl
from pyproj import CRS
from matplotlib import pyplot as plt
import os
import geopandas as gpd
import json

c:\Users\seank\OneDrive\Dokumente\Career Foundry Data Analytics Course\Python_visualisation\CitiBike_NY\venv\Lib\site-packages\keplergl\keplergl.py:13: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_string


In [2]:
df = pd.read_csv(r'C:\Data\Citibike_NY_2022\merged\station_summary.csv')

In [3]:
df.head()

,station_name,lat,lng,no_return_pc,daily_deps,daily_arrs
0,1 Ave & E 110 St,40.792327,-73.938300,-0.7,58.8,59.2
1,1 Ave & E 16 St,40.732219,-73.981656,1.3,184.2,181.9
2,1 Ave & E 18 St,40.733812,-73.980544,0.4,193.8,193.1
3,1 Ave & E 30 St,40.741444,-73.975361,-1.4,124.2,126.0
4,1 Ave & E 39 St,40.747140,-73.971130,-0.8,143.3,144.4


In [4]:
df.shape

(1818, 6)

## Plotting with KeplerGl

In [5]:
df.dtypes

station_name     object
lat             float64
lng             float64
no_return_pc    float64
daily_deps      float64
daily_arrs      float64
dtype: object

In [6]:
print(os.getcwd())

c:\Users\seank\OneDrive\Dokumente\Career Foundry Data Analytics Course\Python_visualisation\CitiBike_NY\notebooks


In [7]:
# load the subway lines GeoJSON 
subway_lines = gpd.read_file("../subway_lines.geojson")

In [8]:
# load subway stations GeoJson
subway_stations = gpd.read_file("../subway_stations.geojson")

In [9]:
print(subway_stations.geometry.geom_type.unique())

['Point']


In [10]:
# load NTA shapes with population and income data merged
ntas = gpd.read_file("C:/Data/Citibike_NY_2022/merged/nta_pop_inc.geojson")
ntas.dtypes

shape_area            object
ntaname               object
cdtaname              object
shape_leng            object
boroname              object
ntatype               object
nta2020               object
borocode              object
countyfips            object
ntaabbrev             object
cdta2020              object
GeoID                 object
median_hh_income     float64
population           float64
area_km2             float64
pop_density          float64
geometry            geometry
dtype: object

## Map with just the top 100 stations and subway

In [11]:
# subsetting top 100 stations
df_100 = df.sort_values(by='daily_deps', ascending=False).head(100).copy()
df_100.head()

,station_name,lat,lng,no_return_pc,daily_deps,daily_arrs
1643,W 21 St & 6 Ave,40.741740,-73.994156,-1.1,352.9,356.7
1773,West St & Chambers St,40.717548,-74.013221,-1.0,337.1,340.6
506,Broadway & W 58 St,40.766953,-73.981693,3.3,312.4,302.2
288,6 Ave & W 33 St,40.749013,-73.988484,1.2,291.1,287.5
8,1 Ave & E 68 St,40.765005,-73.958185,-0.4,286.8,288.0


In [12]:
df_100.describe()

,lat,lng,no_return_pc,daily_deps,daily_arrs
count,100.000000,100.000000,100.000000,100.000000,100.000000
mean,40.742871,-73.989408,-0.266000,197.697000,198.172000
std,0.017715,0.013517,1.409006,40.586567,40.494454
min,40.711444,-74.016584,-5.500000,157.100000,155.400000
25%,40.729760,-73.997252,-0.700000,168.525000,170.300000
50%,40.742889,-73.990426,-0.300000,184.500000,184.300000
75%,40.754974,-73.981597,0.200000,210.375000,210.750000
max,40.779668,-73.953517,4.000000,352.900000,356.700000


In [ ]:
# loading config settings
with open("config_top100.json") as f:
    config = json.load(f)

m_top100 = KeplerGl(height=700, data={"data_100": df_100}, config=config)
m_top100

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'lbngg4c', 'type': …

In [14]:
# save settings
config_top100 = m_top100.config

with open("config_top100.json", "w") as outfile:
    json.dump(config_top100, outfile)

In [15]:
# Saving visualisation
m_top100.save_to_html(file_name = '../visualisations/top100_stations.html', 
               read_only = False, 
               config = config_top100)

Map saved to ../visualisations/top100_stations.html!


### Map with population density as background layer

In [ ]:
# Creating KeplerGL map instance for citibike stations and subway network overlaid with population
# loading saved config
with open("config_pop_dens.json") as f:
    config = json.load(f)

# --- Ensure UI config is present ---
if "uiState" not in config:
    config["uiState"] = {}

# --- Make sure mapControls exist ---
if "mapControls" not in config["uiState"]:
    config["uiState"]["mapControls"] = {}

# --- Force legend + controls visible and active ---
config["uiState"]["mapControls"].update({
    "visibleLayers": True,
    "mapLegend": {"show": True, "active": True},  # ensure legend is visible + expanded
    "toggle3d": False,
    "splitMap": False
})

m_pop = KeplerGl(height=700, data={"data_1": df}, config=config)
m_pop.add_data(subway_lines, name="Subway Lines")
m_pop.add_data(subway_stations, name="Subway Stations")
m_pop.add_data(ntas, name="Income and Population by NTA")
m_pop

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


c:\Users\seank\OneDrive\Dokumente\Career Foundry Data Analytics Course\Python_visualisation\CitiBike_NY\venv\Lib\site-packages\jupyter_client\session.py:721: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant: nan
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)
c:\Users\seank\OneDrive\Dokumente\Career Foundry Data Analytics Course\Python_visualisation\CitiBike_NY\venv\Lib\site-packages\jupyter_client\session.py:721: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant: nan
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)
c:\Users\seank\OneDrive\Dokumente\Career Foundry Data Analytics Course\Python_visualisation\CitiBike_NY\venv\Lib\site-packages\jupyter_client\session.py:721: UserWarning: Message serialization failed with

KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'ozyje6h', 'type': …

In [20]:
# save settings
config_pop = m_pop.config

In [21]:
with open("config_pop_dens.json", "w") as outfile:
    json.dump(config_pop, outfile)

In [22]:
m_pop.save_to_html(file_name = '../visualisations/stops_layers_pop.html', 
               read_only = False, 
               config = config_pop)

Map saved to ../visualisations/stops_layers_pop.html!


### Map with median household income as background layer

In [11]:
# Creating KeplerGL map instance for citibike stations and subway network overlaid with population

with open("config_income.json") as f:
    config = json.load(f)

m_inc = KeplerGl(height=700, data={"data_1": df}, config=config)  
m_inc.add_data(subway_lines, name="Subway Lines")
m_inc.add_data(subway_stations, name="Subway Stations")
m_inc.add_data(ntas, name="Income and Population by NTA")
m_inc

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


c:\Users\seank\OneDrive\Dokumente\Career Foundry Data Analytics Course\Python_visualisation\CitiBike_NY\venv\Lib\site-packages\jupyter_client\session.py:721: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant: nan
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)
c:\Users\seank\OneDrive\Dokumente\Career Foundry Data Analytics Course\Python_visualisation\CitiBike_NY\venv\Lib\site-packages\jupyter_client\session.py:721: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant: nan
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)
c:\Users\seank\OneDrive\Dokumente\Career Foundry Data Analytics Course\Python_visualisation\CitiBike_NY\venv\Lib\site-packages\jupyter_client\session.py:721: UserWarning: Message serialization failed with

KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'ozyje6h', 'type': …

In [21]:
# save settings
config_inc = m_inc.config
with open("config_income.json", "w") as outfile:
    json.dump(config_inc, outfile)

In [22]:
# Saving visualisation
m_inc.save_to_html(file_name="../visualisations/stops_layers_inc.html",
    read_only=False,
    config=config_inc)

Map saved to ../visualisations/stops_layers_inc.html!
